**Mengimport Library yang diperlukan**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import factorial as fact
from math import exp
import os


In [ ]:
def calc_profit(num, sell): #menghitung profit pada tabel di posisi i, j
  res = num*2500
  dif = sell - num
  if dif < 0:
    res = res + dif*8500
  return res

def calc_regret(num, sell): #menghitung regret pada tabel di posisi i, j
  return num - calc_profit(num, sell)

def poisson(num, var): #menghitung poisson proability pada tabel di posisi i
  return (var**num)*exp(-var)/fact(num)

Mengambil data dari google drive dan mengubah data menjadi int

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/Penjualan Nasi.csv')
data.fillna(0, inplace=True)

data['Hari'] = data['Hari'].str.strip()
for col in data.columns[1:]:
  data[col] = data[col].astype(int)
data.head()

,Hari,05.30-06.00,06.00-06.30,06.30-07.00,07.00-07.30,07.30-08.00,08.00-08.30,08.30-09.00,Total
0,0,3,3,6,3,0,0,0,15
1,1,3,1,4,2,0,0,0,10
2,2,4,3,5,0,0,0,0,12
3,3,1,3,4,1,1,0,0,10
4,4,1,4,2,2,0,0,0,9


Menghitung table regret, table profit, serta probabilitas poisson dari 1 hingga 50

In [ ]:
profit_table = {}
regret_table = {}
prob_table = {}

In [ ]:
for col in data.columns[1:-1]:
  profit = np.zeros((50, 50))
  regret = np.zeros((50, 50))
  prob = np.zeros((50))
  for i in range(0, 50):
    for j in range(0, 50):
      profit[i][j] = calc_profit(i, j)
      regret[i][j] = calc_regret(i, j)
    prob[i] = poisson(i, data[col].var())
  profit_table[col] = profit
  regret_table[col] = regret
  prob_table[col] = prob

Menentukan min dan max pada setiap waktu(05.00-05.30, ..., 08.30-09.00) 

In [ ]:
for col in data.columns[1:-1]:
  profit_df = pd.DataFrame(profit_table[col])
  profit_df['min'] = profit_df.min(axis=1)
  profit_df['max'] = profit_df.max(axis=1)
  profit_df.to_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/profit_' + col + '.csv', index=False)

  regret_df = pd.DataFrame(regret_table[col])
  regret_df['min'] = regret_df.min(axis=1)
  regret_df['max'] = regret_df.max(axis=1)
  regret_df.to_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/regret_' + col + '.csv', index=False)

Menghitung profit dengan mengalikan nilai pada table dengan probabilitas yang sesuai dengan posisinya

In [ ]:
expec_profit = {}

In [ ]:
for col in data.columns[1:-1]:
  prft = np.zeros(50)
  for i in range(0, 50):
    prft[i] = np.sum(profit_table[col][i]*prob_table[col])
  expec_profit[col] = prft

pd.DataFrame(expec_profit).to_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/expectation_profit.csv', index=False)

Expectation Criteria

In [ ]:
max_expec_profit = pd.read_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/expectation_profit.csv').idxmax()
print(max_expec_profit)
print('\nTotal', max_expec_profit.sum())

05.30-06.00    1
06.00-06.30    4
06.30-07.00    3
07.00-07.30    3
07.30-08.00    1
08.00-08.30    2
08.30-09.00    0
dtype: int64

Total 14


MAXIMIN PROFIT CRITERIA

In [ ]:
total = 0

for dir in os.listdir('/content/drive/MyDrive/Colab Data/Sains Manajemen')[1:-2]:
  if dir[0] == 'p':
    df = (pd.read_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/'+dir))
    res = df['min'].idxmax()
    print(dir[7:], res)
    total = total + res
print('\nTotal', total)

05.30-06.00.csv 0
06.00-06.30.csv 0
06.30-07.00.csv 0
07.00-07.30.csv 0
07.30-08.00.csv 0
08.00-08.30.csv 0
08.30-09.00.csv 0

Total 0


MAXIMAX PROFIT CRITERIA

In [ ]:
total = 0

for dir in os.listdir('/content/drive/MyDrive/Colab Data/Sains Manajemen')[1:-2]:
  if dir[0] == 'p':
    df = (pd.read_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/'+dir))
    res = df['max'].idxmax()
    print(dir[7:], res)
    total = total + res
print('\nTotal', total)

05.30-06.00.csv 49
06.00-06.30.csv 49
06.30-07.00.csv 49
07.00-07.30.csv 49
07.30-08.00.csv 49
08.00-08.30.csv 49
08.30-09.00.csv 49

Total 343


MINIMAX REGRET CRITERIA

In [ ]:
total = 0

for dir in os.listdir('/content/drive/MyDrive/Colab Data/Sains Manajemen')[1:-2]:
  if dir[0] == 'r':
    df = (pd.read_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/'+dir))
    res = df['max'].idxmin()
    print(dir[7:], res)
    total = total + res
print('\nTotal', total)

05.30-06.00.csv 0
06.00-06.30.csv 0
06.30-07.00.csv 0
07.00-07.30.csv 0
07.30-08.00.csv 0
08.00-08.30.csv 0
08.30-09.00.csv 0

Total 0


FORECASTING

In [ ]:
def forecast(f, d, a): #fungsi forecast
  return a*d + (1-a)*f

def trend(f, pf, pt, b): #menghitung trend
  return b*(f - pf) + (1-b)*pt

def adjust_forecast(f, pf, d, pt, a, b): #menghitung adjust forecast
  return forecast(f, d, a) + trend(f, pf, pt, b)

def combine_arr(X, Y): # menggabungkan 2 array
  c = []

  for x in X:
    for y in Y:
      c.append([x, y])

  return c

membuat parameter alpha dan beta

In [ ]:
parameter_df = pd.DataFrame(combine_arr(np.linspace(0.0, 1.0, 100), np.linspace(0.0, 1.0, 100)), columns=['alpha', 'beta'])
parameter_df.head()

,alpha,beta
0,0.0,0.000000
1,0.0,0.010101
2,0.0,0.020202
3,0.0,0.030303
4,0.0,0.040404


mendapatkan mse dari parameter yang telah dibuat sebelumnya

In [ ]:
for col in data.columns[1:-1]:
  res = []
  for par in range(parameter_df.shape[0]):
    f_data = [data[col].iloc[0], data[col].iloc[1]]
    t_data = [0, 0]
    for i in range(2, data.shape[0]):
      f_data.append(adjust_forecast(f_data[-1], f_data[-2], data[col].iloc[i-1], t_data[-1],
                                    parameter_df['alpha'].iloc[par], parameter_df['beta'].iloc[par]))
    res.append(((data[col] - f_data)**2).mean())
  parameter_df[col] = res
  

menyimpan parameter

In [ ]:
parameter_df.to_csv('/content/drive/MyDrive/Colab Data/Sains Manajemen/forecast_parameter.csv', index=False)

Menetukan parameter yangMSE paling kecil

In [ ]:
parameter_df.idxmin()[2:]

05.30-06.00    1266
06.00-06.30     700
06.30-07.00    1630
07.00-07.30       0
07.30-08.00     198
08.00-08.30     277
08.30-09.00     198
dtype: int64